## machine learning for startup expansion data

## Content
1. Import libraries
2. Load data 
3. Lable encoding
4. Determine features and target
5. Buidling models 
6. Make prediction for a new store revenue

### Import labraries

In [217]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from imblearn.under_sampling import RandomUnderSampler

import imblearn.over_sampling
from imblearn.over_sampling import RandomOverSampler


### Load data

In [218]:
df = pd.read_excel('starup_new.xlsx')

In [219]:
df.head(5)

,Store ID,City,State,Sales Region,New Expansion,Marketing Spend,Revenue,profet
0,1,Peoria,Arizona,Region 2,Old,2601,48610,46009
1,2,Midland,Texas,Region 2,Old,2727,45689,42962
2,3,Spokane,Washington,Region 2,Old,2768,49554,46786
3,4,Denton,Texas,Region 2,Old,2759,38284,35525
4,5,Overland Park,Kansas,Region 2,Old,2869,59887,57018


In [220]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Store ID         150 non-null    int64 
 1   City             150 non-null    object
 2   State            150 non-null    object
 3   Sales Region     150 non-null    object
 4   New Expansion    150 non-null    object
 5   Marketing Spend  150 non-null    int64 
 6   Revenue          150 non-null    int64 
 7   profet           150 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 9.5+ KB


### Label encodig 

In [221]:
label_encoder = LabelEncoder()

In [222]:
df1 = pd.DataFrame()

In [223]:
df1['Store ID'] = df['Store ID']
df1['City'] = label_encoder.fit_transform(df['City'])
df1['State'] = label_encoder.fit_transform(df['State'])
df1['Sales Region'] = label_encoder.fit_transform(df['Sales Region'])
df1['New Expansion'] = label_encoder.fit_transform(df['New Expansion'])
df1['Marketing Spend'] = df['Marketing Spend']
df1['Revenue'] = df['Revenue']
df1['profet'] = df['profet']

In [224]:
df1.head(5)

,Store ID,City,State,Sales Region,New Expansion,Marketing Spend,Revenue,profet
0,1,94,1,1,1,2601,48610,46009
1,2,71,30,1,1,2727,45689,42962
2,3,121,33,1,1,2768,49554,46786
3,4,31,30,1,1,2759,38284,35525
4,5,88,11,1,1,2869,59887,57018


### Determine features and tatrget

In [225]:
x = df1.drop(['Revenue','profet'],axis=1)
y = df1['Revenue']

In [226]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [227]:
X_train.shape

(120, 6)

###  Build machine learning models 

In [228]:
ss = StandardScaler()

X_train_std = ss.fit_transform(X_train)
X_test_std = ss.transform(X_test)

###  1.Logistic regression model

In [229]:
lr =LogisticRegression(n_jobs=5)
lr.fit(X_train_std, y_train)
print("Train Logistic Regression :",lr.score(X_train_std, y_train))
y_pre =lr.predict(X_test_std)

Train Logistic Regression : 0.6583333333333333


In [230]:
y_pred= lr.predict(X_test_std)
df_Acual_pred= pd.DataFrame({'ActualRevenue':y_test,'PredictedRevenue':y_pred})
df_Acual_pred.head(5)

,ActualRevenue,PredictedRevenue
128,59254,19703
11,21988,48610
118,54145,41361
15,16690,48610
123,20703,20669


###  2.Knn model

In [231]:
from sklearn import linear_model, neighbors, ensemble
knn = neighbors.KNeighborsClassifier()
knn.fit(X_train, y_train)
print("Train KNN : ",knn.score(X_train, y_train))

Train KNN :  0.175


In [232]:
y_pred= knn.predict(X_test)
df_Acual_pred= pd.DataFrame({'ActualRevenue':y_test,'PredictedRevenue':y_pred})
df_Acual_pred.head(5)

,ActualRevenue,PredictedRevenue
128,59254,18454
11,21988,16029
118,54145,18215
15,16690,41110
123,20703,21603


###  3. XGBoost model

In [233]:
from xgboost import XGBRegressor
xg = XGBRegressor()
xg.fit(X_train, y_train)
print("Train XGB : ",xg.score(X_train, y_train))

Train XGB :  0.9999999675900239


In [234]:
y_pred= xg.predict(X_test)
df_Acual_pred= pd.DataFrame({'ActualRevenue':y_test,'PredictedRevenue':y_pred})
df_Acual_pred.head(5)

,ActualRevenue,PredictedRevenue
128,59254,47174.835938
11,21988,33610.824219
118,54145,30185.763672
15,16690,44017.316406
123,20703,41945.546875


###  3. Random forest model

In [235]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print("Train Random Forest: ",rf.score(X_train, y_train))

Train Random Forest:  1.0


In [236]:
y_pred= rf.predict(X_test)
df_Acual_pred= pd.DataFrame({'ActualRevenue':y_test,'PredictedRevenue':y_pred})
df_Acual_pred.head(5)

,ActualRevenue,PredictedRevenue
128,59254,19703
11,21988,16860
118,54145,36821
15,16690,16860
123,20703,20669


In [237]:
import joblib

filename = 'finalized_model.sav'
joblib.dump(rf, filename)

['finalized_model.sav']

### Test the model on a New Data

In [238]:
from keras.models import load_model
loaded_model = joblib.load('finalized_model.sav')

# 151 refering to the NEW strore id,88 refering to Overland Park city, 1 refering Kansas state,
# 1 refering to Region 2 and 0 refering to New Expansion,, WITH SAME MARKTING SPEND ^_^

new_store = [[151,88,11,1,0,2869]]

x = pd.DataFrame(new_store, columns=['Store ID','City','State','Sales Region','New Expansion','Marketing Spend'])
X_test_std = ss.transform(x)
print("PREDICTED Revenue for new store :",loaded_model.predict(x))

# returns predicted revenue


PREDICTED Revenue for new store : [43377]


In [239]:
#Revenue of stores in SAME city for the 
df_new = df[df['City'] == 'Overland Park']
df_new

,Store ID,City,State,Sales Region,New Expansion,Marketing Spend,Revenue,profet
4,5,Overland Park,Kansas,Region 2,Old,2869,59887,57018
